In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random

data_folder = "/home/wenceslai/Documents/predict_sales_kaggle"

train_data = pd.read_csv(os.path.join(data_folder, 'sales_train.csv'))
df_train = train_data.copy()

df_itemcats = pd.read_csv(os.path.join(data_folder, 'items.csv'))

df_test = pd.read_csv(os.path.join(data_folder, 'test.csv'))
df_sub = pd.read_csv(os.path.join(data_folder, 'sample_submission.csv'))

print(df_train.shape)
df_train.head()

(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


#correlation matrix
corr = df_train.corr()
sns.heatmap(data=corr, square=True, annot=True, fmt='.2f')

df_train.plot.scatter(x='shop_id', y='item_price')

In [27]:
df_itemcats = df_itemcats.drop('item_name', axis=1)

#aggregating average price and monthly sales
df_train = df_train.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_price' : ['mean'], 'item_cnt_day' : ['sum']})
df_train.columns = ['avg_price', 'month_sum']

df_train = df_train.reset_index()

avg_prices = df_train[['item_id', 'avg_price']].copy()

#adding item categories
df_train = df_train.merge(df_itemcats, how='left')

#df_train['date_block_num'] %= 12

df_train.head()

,date_block_num,shop_id,item_id,avg_price,month_sum,item_category_id
0,0,0,32,221.0,6.0,40
1,0,0,33,347.0,3.0,37
2,0,0,35,247.0,1.0,40
3,0,0,43,221.0,1.0,40
4,0,0,51,128.5,2.0,57


In [28]:
#test data frame
#date_block_num 	shop_id 	item_id 	month_sum 	item_category_id 	item_price

df_test['date_block_num'] = 34 #% 12

df_test = df_test.merge(df_itemcats, on='item_id', how='left')

avg_prices = avg_prices.drop_duplicates() #average prices of items
avg_prices_train = avg_prices.groupby('item_id')['avg_price'].mean()
df_test = df_test.merge(avg_prices_train, on='item_id', how='left')

df_test = df_test.sort_values(by=['ID'])

df_test = df_test[['date_block_num', 'shop_id', 'item_id', 'item_category_id', 'avg_price']]
df_test['avg_price'] = df_test['avg_price'].fillna(df_test['avg_price'].mean())

assert df_test.shape[-1] == df_train.shape[-1] - 1
print(df_test.isnull().sum())

date_block_num      0
shop_id             0
item_id             0
item_category_id    0
avg_price           0
dtype: int64


In [5]:
#adding features: mean encoding, lag features, 

# Training

In [58]:
X = df_train.drop('month_sum', axis=1).values

y = df_train['month_sum'].values
#y = y.reshape(len(y), 1)
y = y.clip(0., 20.)

print(X.shape, y.shape)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8)


(1609124, 5) (1609124,)


In [37]:

val_tresh = 34

X_train = df_train.loc[df_train['date_block_num'] < val_tresh].drop('month_sum', axis=1).values
y_train = df_train.loc[df_train['date_block_num'] < val_tresh]['month_sum'].values


X_val = df_train.loc[df_train['date_block_num'] == val_tresh].drop('month_sum', axis=1).values
y_val = df_train.loc[df_train['date_block_num'] == val_tresh]['month_sum'].values

y_train = y_train.clip(0., 20.)
y_val = y_val.clip(0., 20.)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(1609124, 5) (1609124,) (0, 5) (0,)


In [ ]:
#baseline: 1.41
from sklearn.ensemble import RandomForestRegressor
model_rf = RandomForestRegressor(n_jobs=-1, verbose=1, n_estimators=90, max_depth=15) #rs was 18
model_rf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   42.2s


In [29]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train.ravel(), verbose=1)

[21:31:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

# Validation

In [54]:
from sklearn.metrics import mean_squared_error

y_preds = model_rf.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, y_preds))
print("validation RMSE:", rmse)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s


validation RMSE: 2.1827752680734487


[Parallel(n_jobs=8)]: Done  90 out of  90 | elapsed:    0.4s finished


In [14]:
"""
k = df_train['date_block_num'].nunique()
win_size = X_train.shape[0] // k
rmses = []

for i in range(k - 1):
    X_train_part = X_train[(i + 1) * win_size:(i + 2) * win_size]
    y_train_part = y_train[(i + 1) * win_size:(i + 2) * win_size]
    X_val = X_train[i * win_size:(i + 1) * win_size]
    y_val = y_train[i * win_size:(i + 1) * win_size]

    #model.fit(X_train_part, y_train_part.ravel())
    y_preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_preds))
    rmses.append(rmse)

print(rmses, "\navg rmse:", sum(rmses) / len(rmses))
"""

'\nk = df_train[\'date_block_num\'].nunique()\nwin_size = X_train.shape[0] // k\nrmses = []\n\nfor i in range(k - 1):\n    X_train_part = X_train[(i + 1) * win_size:(i + 2) * win_size]\n    y_train_part = y_train[(i + 1) * win_size:(i + 2) * win_size]\n    X_val = X_train[i * win_size:(i + 1) * win_size]\n    y_val = y_train[i * win_size:(i + 1) * win_size]\n\n    #model.fit(X_train_part, y_train_part.ravel())\n    y_preds = model.predict(X_val)\n    rmse = np.sqrt(mean_squared_error(y_val, y_preds))\n    rmses.append(rmse)\n\nprint(rmses, "\navg rmse:", sum(rmses) / len(rmses))\n'

In [52]:
X_test = df_test.values
print(X_test.shape)

preds = model_rf.predict(X_test)

(214200, 5)


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  90 out of  90 | elapsed:    0.1s finished


In [53]:
df_sub['item_cnt_month'] = np.round(preds)#round?
df_sub.head()
df_sub.to_csv(os.path.join(data_folder, 'sub_out2'), index=False)
print("done")

done
